### Install project

In [1]:
!git clone https://github.com/kornelro/cv_project.git

Cloning into 'cv_project'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (259/259), done.
remote: Total 652 (delta 130), reused 234 (delta 61), pack-reused 278
Receiving objects: 100% (652/652), 103.84 MiB | 22.93 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [2]:
!pip install cv_project/.

Processing ./cv_project
  Created wheel for src: filename=src-0.1.0-cp36-none-any.whl size=16310 sha256=c855b4ddceff272ad33d34eec2ce44734f7b58c40204f9761339d6ca0a1eeaeb
  Stored in directory: /tmp/pip-ephem-wheel-cache-nqdhiyqj/wheels/fc/1b/40/72379bece2bafba26136079aeac87421c9221abafeb9c66746
Successfully built src


### Mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
DRIVE_PATH = '/content/drive/Shareddrives/CV-PROJECT/'

### Imports

In [6]:
from src.data.make_dataset import make_dataset
from src.data.image_loaders import AerialCarsFixedSizeImageLoader
from src.data.dataset_loaders import AerialCarsDatasetLoader
from src.features.pipelines import RawImageToImage
from src.features.processors import NormImage, EqualHist
from src.models.classifiers import ResnetModelWrapper
from src.models.detectors import SliderProbDetector
from src.data.sliders import SlidingWindow
from src.evaluation import validate_model, precision, recall, f1_score

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from copy import deepcopy
import time
import matplotlib.pyplot as plt
import random
from datetime import datetime
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import cv2
from tqdm import tqdm
import pickle
import pandas as pd
import json
from copy import deepcopy

In [53]:
import os
# from shutil import copyfile

# for i in range(1,21):
#   copyfile(DRIVE_PATH+'data/orto_detection/map1_1.txt', DRIVE_PATH+'data/orto_detection/'+str(i)+'/map'+str(i)+'_1.txt')
#   copyfile(DRIVE_PATH+'data/orto_detection/map1_2.txt', DRIVE_PATH+'data/orto_detection/'+str(i)+'/map'+str(i)+'_2.txt')
#   copyfile(DRIVE_PATH+'data/orto_detection/map1_3.txt', DRIVE_PATH+'data/orto_detection/'+str(i)+'/map'+str(i)+'_3.txt')

# for i in range(1,21):
#   copyfile(DRIVE_PATH+'data/raw/orto_parkings/map'+str(i)+'_1.png', DRIVE_PATH+'data/orto_detection/'+str(i)+'/map'+str(i)+'_1.png')
#   copyfile(DRIVE_PATH+'data/raw/orto_parkings/map'+str(i)+'_2.png', DRIVE_PATH+'data/orto_detection/'+str(i)+'/map'+str(i)+'_2.png')
#   copyfile(DRIVE_PATH+'data/raw/orto_parkings/map'+str(i)+'_3.png', DRIVE_PATH+'data/orto_detection/'+str(i)+'/map'+str(i)+'_3.png')

# Parkings

### Train ResNet50

In [ ]:
DATA_PATH = DRIVE_PATH+'data/processed/aerial-cars-dataset/resnet/data_resnet.pickle'
with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

In [ ]:
TEST_SIZE = 0.1
BATCH_SIZE = 100
EPOCHS = 5

In [ ]:
X = np.array(list(map(lambda x: x[1], data)))
y = np.array(list(map(lambda x: x[2], data)))
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=42
  )


input_t = K.Input(shape=(X[0].shape))
res_model = K.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=input_t
)

for layer in res_model.layers:
    layer.trainable = False

model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer=K.optimizers.Adam(0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(X_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS, 
            validation_data=(X_test, y_test)
)

Epoch 1/5
43/43 [==============================] - 8s 88ms/step - loss: 0.3689 - accuracy: 0.8935 - val_loss: 0.1205 - val_accuracy: 0.9767
Epoch 2/5
43/43 [==============================] - 2s 51ms/step - loss: 0.0348 - accuracy: 0.9929 - val_loss: 0.1070 - val_accuracy: 0.9852
Epoch 3/5
43/43 [==============================] - 2s 51ms/step - loss: 0.0050 - accuracy: 0.9981 - val_loss: 0.1052 - val_accuracy: 0.9809
Epoch 4/5
43/43 [==============================] - 2s 50ms/step - loss: 0.0039 - accuracy: 0.9989 - val_loss: 0.0921 - val_accuracy: 0.9831
Epoch 5/5
43/43 [==============================] - 2s 51ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0942 - val_accuracy: 0.9873


### Detection

In [ ]:
del data

In [ ]:
BND_BOX_SIZE = (80, 80)
STEP_SIZE = 20

dataset_loader = AerialCarsDatasetLoader(
    image_loader=AerialCarsFixedSizeImageLoader(
        bnd_box_size=BND_BOX_SIZE
    )
)

process_pipeline=RawImageToImage(
    processors=[
        EqualHist(),
        NormImage()
    ]
)

sliding_window = SlidingWindow(
    step_size=STEP_SIZE,
    window_size=BND_BOX_SIZE
)

slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.4
)

In [ ]:
processed_images_dict = {}

for i in range(1, 21):
  true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/orto_detection/'+str(i),
    output_folder_filepath=DRIVE_PATH+'data/orto_detection/'+str(i),
    images_files_types=('png',),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
  )

  processed_images_dict[i] = processed_images


  0%|          | 0/3 [00:00<?, ?it/s]

346



 67%|██████▋   | 2/3 [00:29<00:14, 14.97s/it]

61
1150



100%|██████████| 3/3 [00:44<00:00, 14.70s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

...


100%|██████████| 3/3 [00:43<00:00, 14.57s/it]

 33%|███▎      | 1/3 [00:14<00:28, 14.37s/it]

268
557



 67%|██████▋   | 2/3 [00:28<00:14, 14.43s/it]

880



100%|██████████| 3/3 [00:43<00:00, 14.51s/it]


In [ ]:
# with open(DRIVE_PATH+'data/orto_parkings/processed_images_dict.pickle', 'wb') as f:
#   pickle.dump(processed_images_dict, f)

### Parkings

#### Load processed images

In [97]:
with open(DRIVE_PATH+'data/orto_parkings/processed_images_dict.pickle', 'rb') as f:
  processed_images_dict = pickle.load(f)

#### Code

In [124]:
def add_points(row):
    c = row['box']
    row['points'] = [
        (c[0], c[1]),
        (c[2], c[3]),
        (c[4], c[5]),
        (c[6], c[7])
        ]

    return row

def validate_parkings(
    eps, min_samples,
    processed_images_dict,
    draw_show_points,
    show_polygons,
    save_polygons,
    show_image_with_mask,
    save_image_with_mask,
    postfix
):

  ious = []

  for kp in range(1, 21):
    print()
    print('Loop', kp)

    masks = []

    for lp in range(3):

      image = processed_images_dict[kp][lp][0]
      bnd_boxes = processed_images_dict[kp][lp][1]

      points_x = list(map(lambda x: x[0] + 40, bnd_boxes))
      points_y = list(map(lambda x: x[5] + 40, bnd_boxes))


      # make clusters

      clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(list(zip(points_x, points_y)))


      if draw_show_points:

        # draw points

        colors = clustering.labels_

        x_to_plot = []
        y_to_plot = []
        c_to_plot = []

        for i in range(len(colors)):
            if colors[i] != -1:
                x_to_plot.append(points_x[i])
                y_to_plot.append(points_y[i])
                c_to_plot.append(colors[i])

        plt.figure(figsize=(20,10))
        plt.imshow(image)
        plt.scatter(x_to_plot, y_to_plot, c=c_to_plot, s=100)
        plt.show()


      # draw polygons

      colors = clustering.labels_

      df = pd.DataFrame(list(zip(points_x, points_y, colors, bnd_boxes)), columns=['x', 'y', 'c', 'box'])
      df = df[df['c'] != -1]

      df = df.apply(add_points, axis=1)

      c_points = {}
      for c in range(df['c'].max()+1):
          p_lists = df[df['c'] == c]['points'].to_list()
          if len(p_lists) > 0:
            c_points[c] = []
            for l in p_lists:
                c_points[c] = c_points[c] + l

      image2 = image
      hulls = []
      for k, v in c_points.items():
          hull = cv2.convexHull(np.array(v))
          hulls.append(hull)
          image2 = cv2.drawContours(image2, [hull], -1, (0, 0, 255), 5)

      if show_polygons:

        # show image
        plt.figure(figsize=(20,10))
        plt.imshow(image2)
        plt.scatter(x_to_plot, y_to_plot, c=c_to_plot, s=70)
        plt.show()

      if save_polygons:

        # save image
        output_full_path = DRIVE_PATH+'data/orto_parkings/'+str(kp)+'/clusters_'+str(lp)+'_eps_'+str(EPS)+'_min_'+str(MIN_SAMPLES)+posfix+'.jpg'
        cv2.imwrite(output_full_path, cv2.cvtColor(image2, cv2.COLOR_RGB2BGR))


      # masks
      mask = np.zeros_like(image)
      cv2.fillPoly(mask, hulls, (0, 0, 255))
      masks.append(mask)


    # ground truth mask

    with open(DRIVE_PATH+'data/orto_parkings/parking-polygon/map'+str(kp)+'_3.json', 'rb') as f:
      p = json.load(f)

    hulls = []

    for annotation in p['items'][0]['annotations']:
      annotated_points = annotation['points']
      processed_points = []
      for i in range(0, len(annotated_points), 2):
        processed_points.append((annotated_points[i], annotated_points[i+1]))
      hull = cv2.convexHull(np.array(processed_points).astype(int))
      hulls.append(hull)

    gt_mask = np.zeros_like(image)
    cv2.fillPoly(gt_mask, hulls, (0, 0, 255))

    
    # masks union

    union = masks[0] & masks[1] & masks[2]
    image_with_mask = cv2.cvtColor(cv2.imread(DRIVE_PATH+'data/orto_parkings/'+str(kp)+'/map'+str(kp)+'_3.png'), cv2.COLOR_RGB2BGR)
    t = [0, 0, 255]


    # count metrics and draw found mask

    gt_points = 0
    mask_points = 0
    common_points = 0

    for i in tqdm(range(union.shape[0])):
        for j in range(union.shape[1]):

            if union[i][j][2]:
              image_with_mask[i][j] = np.array(t)
              mask_points += 1
            
            if gt_mask[i][j][2]:
              gt_points += 1

            if union[i][j][2] and gt_mask[i][j][2]:
              common_points += 1
              mask_points -= 1

    ious.append(common_points / (mask_points + gt_points))

    if show_image_with_mask:

      # show image with mask
      plt.figure(figsize=(20,10))
      plt.imshow(image_with_mask)
      plt.show()

    if save_image_with_mask:

      # save image with mask
      output_full_path = DRIVE_PATH+'data/orto_parkings/'+str(kp)+'/image_with_mask_eps_'+str(EPS)+'_min_'+str(MIN_SAMPLES)+posfix+'.jpg'
      cv2.imwrite(output_full_path, cv2.cvtColor(image_with_mask, cv2.COLOR_RGB2BGR))
    
  return ious

#### Experiments

In [123]:
EPS = 150
MIN_SAMPLES = 3
POSTFIX = ''

ious = validate_parkings(
    eps=EPS,
    min_samples=MIN_SAMPLES,
    processed_images_dict=processed_images_dict,
    draw_show_points=False,
    show_polygons=False,
    save_polygons=False,
    show_image_with_mask=False,
    save_image_with_mask=False,
    posfix=POSTFIX
)

print(ious)
ious_array = np.array(ious)
print('Mean: ', ious_array.mean())
print('Min', ious_array.min())
print('Max', ious_array.max())

[0.057662329753007904, 0.32631643857815024, 0.27334036970024017, 0.0760374109464159, 0.0, 0.06466926133535685, 0.03178284034777655, 0.054081630602197034, 0.07586691673897437, 0.0539604249394965, 0.13261535236056474, 0.079440966027818, 0.044356904316101176, 0.2579655729706972, 0.3856896875683435, 0.082374319820467, 0.1488169012793024, 0.13136534649374207, 0.1399539550351022, 0.24235121248514788]
Mean:  0.1329323920649451
Min 0.0
Max 0.3856896875683435
